In [106]:
import os
import tensorflow as tf


In [107]:
data_dir = os.path.abspath("/Users/mika/Applied-Machine-Learning/Project/dataset/bottle/train/good")
print(data_dir)  # Kontrollera att sökvägen är korrekt

/Users/mika/Applied-Machine-Learning/Project/dataset/bottle/train/good


In [108]:
train_good = [f for f in os.listdir(data_dir) if f.endswith('.png')]

In [109]:
# Directory paths
train_dir = "/Users/mika/Applied-Machine-Learning/Project/dataset/bottle/train"
test_dir = "/Users/mika/Applied-Machine-Learning/Project/dataset/bottle/test" 

# Load train data
train_data = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=(150, 150),
    batch_size=32,
    label_mode='categorical'
)

# Load test data
test_data = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=(150, 150),
    batch_size=32,
    label_mode='categorical'  # Categorical labels for multi-class classification
)

print(f"Class names: {test_data.class_names}")

Found 209 files belonging to 1 classes.
Found 83 files belonging to 4 classes.
Class names: ['broken_large', 'broken_small', 'contamination', 'good']


In [110]:
# Normalize the image data
def normalize_img(image, label):
    return image / 255.0, label

train_data = train_data.map(normalize_img)
test_data = test_data.map(normalize_img)

In [111]:
from tensorflow.keras import layers, models
# Build the CNN model for multi-class classification
def build_cnn_model():
    model = models.Sequential([
        layers.InputLayer(shape=(150, 150, 3)),  # Input shape (150x150x3)
        
        # Convolutional layers with max pooling
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', strides=(2, 2)),
        layers.MaxPooling2D((2, 2)),
        
        layers.Conv2D(64, (3, 3), activation='relu', padding='same', strides=(2, 2)),
        layers.MaxPooling2D((2, 2)),
        
        layers.Conv2D(128, (3, 3), activation='relu', padding='same', strides=(2, 2)),
        layers.MaxPooling2D((2, 2)),
        
        # Flatten the output and pass through dense layers
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(4, activation='softmax')  # 4 output units, one for each class (good, bad, contaminated, etc.)
    ])
    
    # Compile the model with sparse categorical crossentropy loss
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Build the CNN model
cnn_model = build_cnn_model()

In [112]:
cnn_model.fit(
    train_data,
    epochs=10,
    validation_data=test_data
)

Epoch 1/10


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 1), output.shape=(None, 4)

In [ ]:
test_loss, test_accuracy = cnn_model.evaluate(test_data)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")